In [7]:
import pandas as pd
import pickle
import numpy as np
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
import re
import time
from geopy.extra.rate_limiter import RateLimiter

In [8]:
pickle_in = open("parrot.pkl","rb")
links = pickle.load(pickle_in)

In [51]:
name = []
address = []
features = {'Level Access and Automatic Doors (or No Doors)':[], 'Ramped/Sloped Access, and/or Manual Doors':[],
           'Ramped/Sloped Access that is Long / Steep (may have Manual Doors)':[], 'Mobility Impaired Walker':[],
           'On-Site Parking':[], 'Off-Site Parking':[], 'Blue Badge/Accessible Parking Bays':[], 'Designated Drop-Off Point':[],
           'Accessible Toilet':[], 'Ambulant Toilet':[], 'Step Free Standard Toilet':[], 'Gender Neutral Toilet':[],
           'Baby Changing Facility':[], 'Changing Place':[], 'Accessible Changing Room':[], 'Accessible Fitting Room':[],
           'Accessible Accommodation':[], 'Hoist':[], 'Accessible Shower':[], 'Seat(s) Available':[], 'Safe or Quiet Space':[],
           'Sign Language':[], 'Assistive Listening':[], 'Audio Description':[], 'Audio Format Information':[], 'Large Print':[],
           'Braille':[], 'Facilities for Assistance Dogs':[], 'Disability Awareness/Equality Training':[], 'Dementia Friendly':[],
           'Companion/Carer Discount':[], 'Home Visit':[]}

In [52]:
url = links[0]
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tag = type(soup.find('h1', 'u-h2 heading u-colour-primary'))

In [57]:
# find venue details

for i in range(4868,len(links)):
    url = links[i]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    if type(soup.find('h1', 'u-h2 heading u-colour-primary')) == tag:
        name.append(soup.find('h1', 'u-h2 heading u-colour-primary').get_text())
        address.append(soup.find('p').get_text())
    
        page2 = requests.get(url+'/access_symbols/summary')
        soup2 = BeautifulSoup(page2.content, 'html.parser')
    
        temp = []
        for j in range(len(soup2.find_all('span'))):
            temp.append(soup2.find_all('span')[j].get_text().strip())
            
        for k in features.keys():
            if k in temp:
                features[k].append(1)
            else:
                features[k].append(0)
    print(i, 'complete')

4868 complete
4869 complete
4870 complete
4871 complete
4872 complete
4873 complete
4874 complete
4875 complete
4876 complete
4877 complete
4878 complete
4879 complete
4880 complete
4881 complete
4882 complete
4883 complete
4884 complete
4885 complete
4886 complete
4887 complete
4888 complete
4889 complete
4890 complete
4891 complete
4892 complete
4893 complete
4894 complete
4895 complete
4896 complete
4897 complete
4898 complete
4899 complete
4900 complete
4901 complete
4902 complete
4903 complete
4904 complete
4905 complete
4906 complete
4907 complete
4908 complete
4909 complete
4910 complete
4911 complete
4912 complete
4913 complete
4914 complete
4915 complete
4916 complete
4917 complete
4918 complete
4919 complete
4920 complete
4921 complete
4922 complete
4923 complete
4924 complete
4925 complete
4926 complete
4927 complete
4928 complete
4929 complete
4930 complete
4931 complete
4932 complete
4933 complete
4934 complete
4935 complete
4936 complete
4937 complete
4938 complete
4939 c

5454 complete
5455 complete
5456 complete
5457 complete
5458 complete
5459 complete
5460 complete
5461 complete
5462 complete
5463 complete
5464 complete
5465 complete
5466 complete
5467 complete
5468 complete
5469 complete
5470 complete
5471 complete
5472 complete
5473 complete
5474 complete
5475 complete
5476 complete
5477 complete
5478 complete
5479 complete
5480 complete
5481 complete
5482 complete
5483 complete
5484 complete
5485 complete
5486 complete
5487 complete
5488 complete
5489 complete
5490 complete
5491 complete
5492 complete
5493 complete
5494 complete
5495 complete
5496 complete
5497 complete
5498 complete
5499 complete
5500 complete
5501 complete
5502 complete
5503 complete
5504 complete
5505 complete
5506 complete
5507 complete
5508 complete
5509 complete
5510 complete
5511 complete
5512 complete
5513 complete
5514 complete
5515 complete
5516 complete
5517 complete
5518 complete
5519 complete
5520 complete
5521 complete
5522 complete
5523 complete
5524 complete
5525 c

6040 complete
6041 complete
6042 complete
6043 complete
6044 complete
6045 complete
6046 complete
6047 complete
6048 complete
6049 complete
6050 complete
6051 complete
6052 complete
6053 complete
6054 complete
6055 complete
6056 complete
6057 complete
6058 complete
6059 complete
6060 complete
6061 complete
6062 complete
6063 complete
6064 complete
6065 complete
6066 complete
6067 complete
6068 complete
6069 complete
6070 complete
6071 complete
6072 complete
6073 complete
6074 complete
6075 complete
6076 complete
6077 complete
6078 complete
6079 complete
6080 complete
6081 complete
6082 complete
6083 complete
6084 complete
6085 complete
6086 complete
6087 complete
6088 complete
6089 complete
6090 complete
6091 complete
6092 complete
6093 complete
6094 complete
6095 complete
6096 complete
6097 complete
6098 complete
6099 complete
6100 complete
6101 complete
6102 complete
6103 complete
6104 complete
6105 complete
6106 complete
6107 complete
6108 complete
6109 complete
6110 complete
6111 c

6626 complete
6627 complete
6628 complete
6629 complete
6630 complete
6631 complete
6632 complete
6633 complete
6634 complete
6635 complete
6636 complete
6637 complete
6638 complete
6639 complete
6640 complete
6641 complete
6642 complete
6643 complete
6644 complete
6645 complete
6646 complete
6647 complete
6648 complete
6649 complete
6650 complete
6651 complete
6652 complete
6653 complete
6654 complete
6655 complete
6656 complete
6657 complete
6658 complete
6659 complete
6660 complete
6661 complete
6662 complete
6663 complete
6664 complete
6665 complete
6666 complete
6667 complete
6668 complete
6669 complete
6670 complete
6671 complete
6672 complete
6673 complete
6674 complete
6675 complete
6676 complete
6677 complete
6678 complete
6679 complete
6680 complete
6681 complete
6682 complete
6683 complete
6684 complete
6685 complete
6686 complete
6687 complete
6688 complete
6689 complete
6690 complete
6691 complete
6692 complete
6693 complete
6694 complete
6695 complete
6696 complete
6697 c

7212 complete
7213 complete
7214 complete
7215 complete
7216 complete
7217 complete
7218 complete
7219 complete
7220 complete
7221 complete
7222 complete
7223 complete
7224 complete
7225 complete
7226 complete
7227 complete
7228 complete
7229 complete
7230 complete
7231 complete
7232 complete
7233 complete
7234 complete
7235 complete
7236 complete
7237 complete
7238 complete
7239 complete
7240 complete
7241 complete
7242 complete
7243 complete
7244 complete
7245 complete
7246 complete
7247 complete
7248 complete
7249 complete
7250 complete
7251 complete
7252 complete
7253 complete
7254 complete
7255 complete
7256 complete
7257 complete
7258 complete
7259 complete
7260 complete
7261 complete
7262 complete
7263 complete
7264 complete
7265 complete
7266 complete
7267 complete
7268 complete
7269 complete
7270 complete
7271 complete
7272 complete
7273 complete
7274 complete
7275 complete
7276 complete
7277 complete
7278 complete
7279 complete
7280 complete
7281 complete
7282 complete
7283 c

...

34747 complete
34748 complete
34749 complete
34750 complete
34751 complete
34752 complete
34753 complete
34754 complete
34755 complete
34756 complete
34757 complete
34758 complete
34759 complete
34760 complete
34761 complete
34762 complete
34763 complete
34764 complete
34765 complete
34766 complete
34767 complete
34768 complete
34769 complete
34770 complete
34771 complete
34772 complete
34773 complete
34774 complete
34775 complete
34776 complete
34777 complete
34778 complete
34779 complete
34780 complete
34781 complete
34782 complete
34783 complete
34784 complete
34785 complete
34786 complete
34787 complete
34788 complete
34789 complete
34790 complete
34791 complete
34792 complete
34793 complete
34794 complete
34795 complete
34796 complete
34797 complete
34798 complete
34799 complete
34800 complete
34801 complete
34802 complete
34803 complete
34804 complete
34805 complete
34806 complete
34807 complete
34808 complete
34809 complete
34810 complete
34811 complete
34812 complete
34813 comp

35294 complete
35295 complete
35296 complete
35297 complete
35298 complete
35299 complete
35300 complete
35301 complete
35302 complete
35303 complete
35304 complete
35305 complete
35306 complete
35307 complete
35308 complete
35309 complete
35310 complete
35311 complete
35312 complete
35313 complete
35314 complete
35315 complete
35316 complete
35317 complete
35318 complete
35319 complete
35320 complete
35321 complete
35322 complete
35323 complete
35324 complete
35325 complete
35326 complete
35327 complete
35328 complete
35329 complete
35330 complete
35331 complete
35332 complete
35333 complete
35334 complete
35335 complete
35336 complete
35337 complete
35338 complete
35339 complete
35340 complete
35341 complete
35342 complete
35343 complete
35344 complete
35345 complete
35346 complete
35347 complete
35348 complete
35349 complete
35350 complete
35351 complete
35352 complete
35353 complete
35354 complete
35355 complete
35356 complete
35357 complete
35358 complete
35359 complete
35360 comp

35841 complete
35842 complete
35843 complete
35844 complete
35845 complete
35846 complete
35847 complete
35848 complete
35849 complete
35850 complete
35851 complete
35852 complete
35853 complete
35854 complete
35855 complete
35856 complete
35857 complete
35858 complete
35859 complete
35860 complete
35861 complete
35862 complete
35863 complete
35864 complete
35865 complete
35866 complete
35867 complete
35868 complete
35869 complete
35870 complete
35871 complete
35872 complete
35873 complete
35874 complete
35875 complete
35876 complete
35877 complete
35878 complete
35879 complete
35880 complete
35881 complete
35882 complete
35883 complete
35884 complete
35885 complete
35886 complete
35887 complete
35888 complete
35889 complete
35890 complete
35891 complete
35892 complete
35893 complete
35894 complete
35895 complete
35896 complete
35897 complete
35898 complete
35899 complete
35900 complete
35901 complete
35902 complete
35903 complete
35904 complete
35905 complete
35906 complete
35907 comp

36388 complete
36389 complete
36390 complete
36391 complete
36392 complete
36393 complete
36394 complete
36395 complete
36396 complete
36397 complete
36398 complete
36399 complete
36400 complete
36401 complete
36402 complete
36403 complete
36404 complete
36405 complete
36406 complete
36407 complete
36408 complete
36409 complete
36410 complete
36411 complete
36412 complete
36413 complete
36414 complete
36415 complete
36416 complete
36417 complete
36418 complete
36419 complete
36420 complete
36421 complete
36422 complete
36423 complete
36424 complete
36425 complete
36426 complete
36427 complete
36428 complete
36429 complete
36430 complete
36431 complete
36432 complete
36433 complete
36434 complete
36435 complete
36436 complete
36437 complete
36438 complete
36439 complete
36440 complete
36441 complete
36442 complete
36443 complete
36444 complete
36445 complete
36446 complete
36447 complete
36448 complete
36449 complete
36450 complete
36451 complete
36452 complete
36453 complete
36454 comp

36935 complete
36936 complete
36937 complete
36938 complete
36939 complete
36940 complete
36941 complete
36942 complete
36943 complete
36944 complete
36945 complete
36946 complete
36947 complete
36948 complete
36949 complete
36950 complete
36951 complete
36952 complete
36953 complete
36954 complete
36955 complete
36956 complete
36957 complete
36958 complete
36959 complete
36960 complete
36961 complete
36962 complete
36963 complete
36964 complete
36965 complete
36966 complete
36967 complete
36968 complete
36969 complete
36970 complete
36971 complete
36972 complete
36973 complete
36974 complete
36975 complete
36976 complete
36977 complete
36978 complete
36979 complete
36980 complete
36981 complete
36982 complete
36983 complete
36984 complete
36985 complete
36986 complete
36987 complete
36988 complete
36989 complete
36990 complete
36991 complete
36992 complete
36993 complete
36994 complete
36995 complete
36996 complete
36997 complete
36998 complete
36999 complete
37000 complete
37001 comp

In [58]:
len(name)

36558

In [59]:
len(address)

36558

In [60]:
len()

37460

In [61]:
df = pd.DataFrame({'Name': name, 'Address': address, 'Full': [m+', '+str(n) for m,n in zip(name,address)]})
df = df.join(pd.DataFrame.from_dict(features))

In [64]:
pickle_out = open("df.pickle","wb")
pickle.dump(df, pickle_out)
pickle_out.close()

pickle_out = open("name.pickle","wb")
pickle.dump(name, pickle_out)
pickle_out.close()

pickle_out = open("address.pickle","wb")
pickle.dump(address, pickle_out)
pickle_out.close()

pickle_out = open("features.pickle","wb")
pickle.dump(features, pickle_out)
pickle_out.close()

In [63]:
df

,Name,Address,Full,Level Access and Automatic Doors (or No Doors),"Ramped/Sloped Access, and/or Manual Doors",Ramped/Sloped Access that is Long / Steep (may have Manual Doors),Mobility Impaired Walker,On-Site Parking,Off-Site Parking,Blue Badge/Accessible Parking Bays,...,Assistive Listening,Audio Description,Audio Format Information,Large Print,Braille,Facilities for Assistance Dogs,Disability Awareness/Equality Training,Dementia Friendly,Companion/Carer Discount,Home Visit
0,All Saints Church,"High Street, Knapwell, Cambridgeshire, CB23 4NW","All Saints Church, High Street, Knapwell, Camb...",0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Ely Museum,"The Old Gaol, Market Street, Ely, Cambridgeshi...","Ely Museum, The Old Gaol, Market Street, Ely, ...",0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,King's College Visitor Centre,"13 King's Parade, Cambridge, Cambridgeshire, C...","King's College Visitor Centre, 13 King's Parad...",0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Milton Country Park,"Cambridge Road, Milton, Cambridgeshire, CB24 6AZ","Milton Country Park, Cambridge Road, Milton, C...",0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Nene Valley Railway - Platforms and Grounds,"Wansford Station, Old Great North Road, Stibbi...","Nene Valley Railway - Platforms and Grounds, W...",0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Tate Modern,"Bankside, London, SE1 9TG","Tate Modern, Bankside, London, SE1 9TG",1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
6,Tower of London,"Tower Hill, London, EC3N 4AB","Tower of London, Tower Hill, London, EC3N 4AB",0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,Belfast Castle,"Antrim Road, Belfast, BT15 5GR","Belfast Castle, Antrim Road, Belfast, BT15 5GR",0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Crumlin Road Gaol,"53-55 Crumlin Road, Belfast, BT14 6ST","Crumlin Road Gaol, 53-55 Crumlin Road, Belfast...",0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,SS Nomadic,"Hamilton Dock, Queens Road, Belfast, BT3 9DT","SS Nomadic, Hamilton Dock, Queens Road, Belfas...",0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
pickle_in = open("df.pickle","rb")
df = pickle.load(pickle_in)

In [37]:
latitude = []
longitude = []

'0.31 IBERS, IBERS New Build, Aberystwyth University, Penglais Campus, Aberystwyth, SY23 3DA'

In [204]:
geolocator = Nominatim(timeout=5)

for i in range(11790, len(df)):
    location = geolocator.geocode(df['Full'][i])
    if location is None:
        a = re.split('[?.,\-!+()]', df['Full'][i])
        location2 = geolocator.geocode(a[0]+', '+a[-1])
        if location2 is None:
            latitude.append('NA')
            longitude.append('NA')
        else:
            latitude.append(location2.latitude)
            longitude.append(location2.longitude)      
    else:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    print(i, 'complete')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [205]:
i

11790

In [206]:
pickle_out = open("longitude.pickle","wb")
pickle.dump(longitude, pickle_out)
pickle_out.close()

pickle_out = open("latitude.pickle","wb")
pickle.dump(latitude, pickle_out)
pickle_out.close()